<a href="https://colab.research.google.com/github/YujiK-github/kaggle_LECR/blob/main/1st_stage_model/1st_stage_model_train_exp006_fold0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Thu Mar  2 15:47:01 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    50W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U sentence-transformers
!pip install datasets transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 86.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 46.1 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=bb9b5c225fd75269b7ff810c21b52db39ab8f3a241cc5ed16a643db620a10899
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
%%writefile 1st_stage_model_train.py

# ===============================================================
#  Library
# ===============================================================
import os
import re
import gc
import sys
import math
import copy
import glob
import random
import argparse
import scipy as sp
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter("ignore")

from sklearn.model_selection import GroupKFold
import heapq

import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from datasets import Dataset


import transformers
from transformers import AutoModel, AutoTokenizer, AutoConfig, DataCollatorWithPadding
transformers.logging.set_verbosity_error()

from sentence_transformers import SentenceTransformer, models, InputExample, losses, evaluation, datasets

device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)

import json
import logging
import os
import time
import shutil
import stat
from collections import OrderedDict
from typing import List, Dict, Tuple, Iterable, Type, Union, Callable, Optional
import requests
from sentence_transformers import __MODEL_HUB_ORGANIZATION__
from sentence_transformers.evaluation import SentenceEvaluator
from sentence_transformers.util import import_from_string, batch_to_device, fullname, snapshot_download
from sentence_transformers.models import Transformer, Pooling, Dense
from sentence_transformers.model_card_templates import ModelCardTemplate
from sentence_transformers import __version__
from torch.optim import Optimizer

import json
import logging
import os
import shutil
import stat
from collections import OrderedDict
from typing import List, Dict, Tuple, Iterable, Type, Union, Callable, Optional
import requests
import numpy as np
from numpy import ndarray
import transformers
from huggingface_hub import HfApi, HfFolder, Repository, hf_hub_url, cached_download
import torch
from torch import nn, Tensor, device
from torch.optim import Optimizer
from torch.utils.data import DataLoader
import torch.multiprocessing as mp
from tqdm.autonotebook import trange
import math
import queue
import tempfile
from distutils.dir_util import copy_tree

# ===============================================================
#  args
# ===============================================================
def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--seed", type=int, default=42, required=False)
    parser.add_argument("--input_dir", type=str, 
                        default="/content/drive/MyDrive/KAGGLE-LECR/", 
                        required=False)
    parser.add_argument("--output_dir", type=str, default="/content/drive/MyDrive/KAGGLE-LECR/", required=False)
    parser.add_argument("--model", type=str, 
                        default="sentence-transformers/all-mpnet-base-v2", required=False)
    parser.add_argument("--batch_size", type=int, default=768, required=False)
    parser.add_argument("--max_len", type=int, default=128, required=False)
    parser.add_argument("--epochs", type=int, default=30, required=False)
    parser.add_argument("--trn_fold", type=int, default=0, required=False)
    parser.add_argument("--n_splits", type=int, default=3, required=False)
    parser.add_argument("--print_freq", type=int, default=200, required=False)
    parser.add_argument("--steps_per_epoch", type=int, default=None, required=False)
    parser.add_argument("--patience", type=int, default=3, required=False)
    parser.add_argument("--filename", type=str, default="exp006", required=False)
    return parser.parse_args()


# ===============================================================
#  Utils
# ===============================================================

import json
import requests

# 任意のメッセージを通知する関数
def send_slack_message_notification(message):
    webhook_url = ' [URL] '  
    data = json.dumps({'text': message})
    headers = {'content-type': 'application/json'}
    requests.post(webhook_url, data=data, headers=headers)
    
    
def seed_everything(cfg):
    """set seed"""
    random.seed(cfg.seed)
    os.environ['PYTHONHASHSEED'] = str(cfg.seed)
    np.random.seed(cfg.seed)
    torch.manual_seed(cfg.seed)
    torch.cuda.manual_seed(cfg.seed)
    torch.backends.cudnn.deterministic = True
    

def data_load(cfg):
    """loading data, rename columns, fillna text"""
    print("========== Data Loading ==========")
    df_content = pd.read_csv(cfg.input_dir+"content.csv").fillna({"title": "", "description": "", "text":""})
    df_topics = pd.read_csv(cfg.input_dir+"topics.csv").fillna({"title": "", "description": ""})
    df_corr = pd.read_csv(cfg.input_dir+"correlations.csv")   
    
    df_content.rename(columns={"id":"content_id"}, inplace=True)
    df_topics.rename(columns={"id":"topic_id"}, inplace=True)
    df_corr.rename(columns={"content_ids":"content_id"}, inplace=True)
    
    cfg.tokenizer = AutoTokenizer.from_pretrained(cfg.model, is_fast=True)
    
    df_content["content_sentence"] = df_content["title"] + cfg.tokenizer.sep_token + df_content["description"]
    df_topics["topic_sentence"] = df_topics["title"] + cfg.tokenizer.sep_token +  df_topics["description"] +\
    cfg.tokenizer.sep_token + df_topics["context"]
    df_topics["topic_sentence"] = df_topics["topic_sentence"].str.replace(" >> ",  " ")
    df_topics = pd.merge(df_topics, df_corr, on="topic_id", how="left")
    
    
    df_topics["content_id"] = df_topics["content_id"].str.split()
    df_topics = df_topics.explode("content_id", ignore_index=True)
    
    df_topics = pd.merge(df_topics, df_content[["content_id", "content_sentence"]], on="content_id", how="left")
    
    print(df_topics.shape)
    # category == sourceのtopic
    df_train = df_topics[(df_topics["category"] == "source")&(df_topics["has_content"] == True)].reset_index(drop=True)
    
    # category == 
    df_valid = df_topics[(df_topics["category"] != "source")&(df_topics["has_content"] == True)].reset_index(drop=True)
    
    
    """
    split cv using GroupKFold
    - split based on number of target's content_id or topic_id
    """
    print("========== CV split ==========")
    gkf = GroupKFold(cfg.n_splits)
    for i, (tr, val) in enumerate(gkf.split(X=df_valid, groups=df_valid["channel"])):
        df_valid.loc[val, "fold"] = i
    print(df_valid.groupby("fold").size())
    print(df_valid[["topic_id", "fold"]].drop_duplicates().groupby("fold").size())
    
    _df_train = df_valid[df_valid["fold"] != cfg.trn_fold].reset_index(drop=True)
    df_valid = df_valid[df_valid["fold"] == cfg.trn_fold].reset_index(drop=True)
    
    df_train = pd.concat([df_train, _df_train], ignore_index=True)

    print("df_content", df_content.shape)
    print("df_corr", df_corr.shape)
    print("df_train", df_train.shape)
    print("df_valid", df_valid.shape)
    
    print("Input Sentence Example")
    print("========== Topics ==========")
    print(df_topics["topic_sentence"].values.tolist()[:2])
    print("========== Content ==========")
    print(df_content["content_sentence"].values.tolist()[:2])
    print(set(df_valid["channel"].values.tolist())&set(df_train["channel"].values.tolist()))
    return df_content, df_train, df_valid

# ===============================================================
#  Prepare_df
# ===============================================================
def prepare_train(train):
    train = train
    train_df = train[["topic_sentence", "content_sentence"]]
    dataset = Dataset.from_pandas(train_df)
    print(dataset)
    train_examples = []
    n_examples = dataset.num_rows
    for i in tqdm(range(n_examples)):
        example = dataset[i]
        if example["topic_sentence"] == None or example["content_sentence"] == None: #remove None
            print(example)
            continue        
        train_examples.append(InputExample(texts=[str(example["topic_sentence"]), str(example["content_sentence"])]))
    return train_examples

def prepare_valid(df_content, df_valid):
    """
    Create a query and corpus like the folloing.
    
    ex)
    queries = {'q1': 'What is machine learning?',
               'q2': 'How does deep learning work?'}
    corpus = {'d1': 'Machine learning is a method of data analysis.', 
              'd2': 'Deep learning is a subfield of machine learning.', 
              'd3': 'Neural networks are used in deep learning.'}
    relevant_docs = {'q1': {'d1'}, 
                     'q2': {'d2', 'd3'}}

    evaluator = evaluation.InformationRetrievalEvaluator(queries, corpus, relevant_docs)
    """
    queries = df_valid[["topic_id", "topic_sentence"]].set_index('topic_id').to_dict()['topic_sentence']
    corpus = df_content[["content_id", "content_sentence"]].set_index('content_id').to_dict()['content_sentence']
    relevant_docs =  df_valid.groupby('topic_id')['content_id'].apply(set).to_dict()    
    return queries, corpus, relevant_docs

def prepare_df(df_content, df_train, df_valid):
    train_examples = prepare_train(df_train)
    queries, corpus, relevant_docs = prepare_valid(df_content, df_valid)
    return train_examples, queries, corpus, relevant_docs

# ===============================================================
# Helper functions
# ===============================================================
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))

def cos_sim(a: Tensor, b: Tensor):
    """
    Computes the cosine similarity cos_sim(a[i], b[j]) for all i and j.
    :return: Matrix with res[i][j]  = cos_sim(a[i], b[j])
    """
    if not isinstance(a, torch.Tensor):
        a = torch.tensor(a)

    if not isinstance(b, torch.Tensor):
        b = torch.tensor(b)

    if len(a.shape) == 1:
        a = a.unsqueeze(0)

    if len(b.shape) == 1:
        b = b.unsqueeze(0)

    a_norm = torch.nn.functional.normalize(a, p=2, dim=1)
    b_norm = torch.nn.functional.normalize(b, p=2, dim=1)
    #print(a_norm.shape)
    #print(b_norm.shape)
    return torch.mm(a_norm, b_norm.transpose(0, 1))


def dot_score(a: Tensor, b: Tensor):
    """
    Computes the dot-product dot_prod(a[i], b[j]) for all i and j.
    :return: Matrix with res[i][j]  = dot_prod(a[i], b[j])
    """
    if not isinstance(a, torch.Tensor):
        a = torch.tensor(a)

    if not isinstance(b, torch.Tensor):
        b = torch.tensor(b)

    if len(a.shape) == 1:
        a = a.unsqueeze(0)

    if len(b.shape) == 1:
        b = b.unsqueeze(0)
    #print(a.shape)
    #print(b.shape)
    return torch.mm(a, b.transpose(0, 1))


class EarlyStopping:
    """
    earlystoppingクラス
    ref: https://qiita.com/ku_a_i/items/ba33c9ce3449da23b503
    """

    def __init__(self, patience=3):
        """引数：最小値の非更新数カウンタ、表示設定、モデル格納path"""

        self.patience = patience    #設定ストップカウンタ
        self.counter = 0            #現在のカウンタ値
        self.best_score = None      #ベストスコア
        self.early_stop = False     #ストップフラグ

    def __call__(self, score, epoch, steps):
        """
        特殊(call)メソッド
        実際に学習ループ内で最小lossを更新したか否かを計算させる部分
        """
        if self.best_score is None:  #1Epoch目の処理
            self.best_score = score   #1Epoch目はそのままベストスコアとして記録する
        elif score < self.best_score:  # ベストスコアを更新できなかった場合
            self.counter += 1   #ストップカウンタを+1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')  #現在のカウンタを表示する 
            send_slack_message_notification(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:  #設定カウントを上回ったらストップフラグをTrueに変更
                self.early_stop = True
        else:  #ベストスコアを更新した場合
            self.best_score = score  #ベストスコアを上書き
            self.counter = 0  #ストップカウンタリセット

# ===============================================================
#  CustomTransformer
# ===============================================================
class CustomTransformer(SentenceTransformer):
    def fit(self,
            cfg,
            train_objectives: Iterable[Tuple[DataLoader, nn.Module]],
            evaluator: SentenceEvaluator = None,
            epochs: int = 1,
            steps_per_epoch = None,
            scheduler: str = 'WarmupLinear',
            warmup_steps: int = 10000,
            optimizer_class: Type[Optimizer] = torch.optim.AdamW,
            optimizer_params : Dict[str, object]= {'lr': 2e-5},
            weight_decay: float = 0.01,
            evaluation_steps: int = 0,
            output_path: str = None,
            save_best_model: bool = True,
            max_grad_norm: float = 1,
            use_amp: bool = False,
            callback: Callable[[float, int, int], None] = None,
            show_progress_bar: bool = True,
            checkpoint_path: str = None,
            checkpoint_save_steps: int = 500,
            checkpoint_save_total_limit: int = 0
            ):
        """
        クラスの継承
        変更点
        * tqdmの中にtqdmを入れるのをなくした
        * scoreが表示されるようにした
        """
        
        info_loss_functions =  []
        for dataloader, loss in train_objectives:
            info_loss_functions.extend(ModelCardTemplate.get_train_objective_info(dataloader, loss))
        info_loss_functions = "\n\n".join([text for text in info_loss_functions])

        info_fit_parameters = json.dumps({"evaluator": fullname(evaluator), "epochs": epochs, "steps_per_epoch": steps_per_epoch, "scheduler": scheduler, "warmup_steps": warmup_steps, "optimizer_class": str(optimizer_class),  "optimizer_params": optimizer_params, "weight_decay": weight_decay, "evaluation_steps": evaluation_steps, "max_grad_norm": max_grad_norm }, indent=4, sort_keys=True)
        self._model_card_text = None
        self._model_card_vars['{TRAINING_SECTION}'] = ModelCardTemplate.__TRAINING_SECTION__.replace("{LOSS_FUNCTIONS}", info_loss_functions).replace("{FIT_PARAMETERS}", info_fit_parameters)


        if use_amp:
            from torch.cuda.amp import autocast
            scaler = torch.cuda.amp.GradScaler()

        self.to(self._target_device)

        dataloaders = [dataloader for dataloader, _ in train_objectives]

        # Use smart batching
        for dataloader in dataloaders:
            dataloader.collate_fn = self.smart_batching_collate

        loss_models = [loss for _, loss in train_objectives]
        for loss_model in loss_models:
            loss_model.to(self._target_device)

        self.best_score = -9999999

        if steps_per_epoch is None or steps_per_epoch == 0:
            steps_per_epoch = min([len(dataloader) for dataloader in dataloaders])

        num_train_steps = int(steps_per_epoch * epochs)

        # Prepare optimizers
        optimizers = []
        schedulers = []
        for loss_model in loss_models:
            param_optimizer = list(loss_model.named_parameters())

            no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
            optimizer_grouped_parameters = [
                {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': weight_decay},
                {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
            ]

            optimizer = optimizer_class(optimizer_grouped_parameters, **optimizer_params)
            scheduler_obj = self._get_scheduler(optimizer, scheduler=scheduler, warmup_steps=warmup_steps, t_total=num_train_steps)

            optimizers.append(optimizer)
            schedulers.append(scheduler_obj)


        global_step = 0
        data_iterators = [iter(dataloader) for dataloader in dataloaders]

        num_train_objectives = len(train_objectives)
        skip_scheduler = False
        for epoch in range(epochs):
            losses = AverageMeter()
            training_steps = 0
            if epoch == 0:
                # self._eval_during_training(evaluator, output_path, save_best_model, -1, training_steps, callback)
                pass
            start = end = time.time()

            for loss_model in loss_models:
                loss_model.zero_grad()
                loss_model.train()

            for step in trange(steps_per_epoch, desc="Iteration", smoothing=0.05, disable=not show_progress_bar):
                for train_idx in range(num_train_objectives):
                    loss_model = loss_models[train_idx]
                    optimizer = optimizers[train_idx]
                    scheduler = schedulers[train_idx]
                    data_iterator = data_iterators[train_idx]

                    try:
                        data = next(data_iterator)
                    except StopIteration:
                        data_iterator = iter(dataloaders[train_idx])
                        data_iterators[train_idx] = data_iterator
                        data = next(data_iterator)

                    features, labels = data
                    labels = labels.to(self._target_device)
                    batch_size = labels.size(0)
                    features = list(map(lambda batch: batch_to_device(batch, self._target_device), features))
               
                    if use_amp:
                        with autocast():
                            loss_value = loss_model(features, labels)
                        losses.update(loss_value.item(), batch_size)

                        scale_before_step = scaler.get_scale()
                        scaler.scale(loss_value).backward()
                        scaler.unscale_(optimizer)
                        grad_norm = torch.nn.utils.clip_grad_norm_(loss_model.parameters(), max_grad_norm)
                        scaler.step(optimizer)
                        scaler.update()

                        skip_scheduler = scaler.get_scale() != scale_before_step
                    else:
                        loss_value = loss_model(features, labels)
                        losses.update(loss_value.item(), batch_size)
                        loss_value.backward()
                        grad_norm = torch.nn.utils.clip_grad_norm_(loss_model.parameters(), max_grad_norm)
                        optimizer.step()

                    optimizer.zero_grad()

                    if not skip_scheduler:
                        scheduler.step()
                    end = time.time()
                    if step % cfg.print_freq == 0 or step == (steps_per_epoch-1):
                        print('Epoch: [{0}][{1}/{2}] '
                              'Elapsed {remain:s} '
                              'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                              'Grad: {grad_norm:.4f}  '
                              'LR: {lr:.8f}  '
                              .format(epoch+1, step, steps_per_epoch, 
                                      remain=timeSince(start, float(step+1)/steps_per_epoch),
                                      loss=losses,
                                      grad_norm=grad_norm,
                                      lr=scheduler.get_lr()[0]))

                training_steps += 1
                global_step += 1


                if evaluation_steps > 0 and training_steps % evaluation_steps == 0:
                    self._eval_during_training(evaluator, output_path, save_best_model, epoch, training_steps, callback)

                    for loss_model in loss_models:
                        loss_model.zero_grad()
                        loss_model.train()

                if checkpoint_path is not None and checkpoint_save_steps is not None and checkpoint_save_steps > 0 and global_step % checkpoint_save_steps == 0:
                    self._save_checkpoint(checkpoint_path, checkpoint_save_total_limit, global_step)

            self._eval_during_training(evaluator, output_path, save_best_model, epoch, -1, callback)

        if evaluator is None and output_path is not None:   #No evaluator, but output path: save final model version
            self.save(output_path)

        if checkpoint_path is not None:
            self._save_checkpoint(checkpoint_path, checkpoint_save_total_limit, global_step)
    
    def _eval_during_training(self, evaluator, output_path, save_best_model, epoch, steps, callback):
        """Runs evaluation during the training"""
        eval_path = output_path
        if output_path is not None:
            os.makedirs(output_path, exist_ok=True)
            eval_path = os.path.join(output_path, "eval")
            os.makedirs(eval_path, exist_ok=True)

        if evaluator is not None:
            score = evaluator(self, output_path=eval_path, epoch=epoch, steps=steps)
            send_slack_message_notification(f"Epoch: [{epoch+1}] recall@k: {score}")
            print(f"Epoch: [{epoch+1}] recall@k:", score)
            if callback is not None:
                callback(score, epoch, steps)
                if callback.early_stop:
                    print("Early Stopping!")
                    send_slack_message_notification(f"Early Stopping!")
                    sys.exit()
            if score > self.best_score:
                self.best_score = score
                print('\033[32m'+f"Save the best model: score {self.best_score}"+'\033[0m')
                if save_best_model:
                    self.save(output_path)
    
# ===============================================================
#  CustomEvaluation
# ===============================================================
class CustomEvaluator(evaluation.InformationRetrievalEvaluator):
    """
    クラスの継承
    
    map@k -> recall@kに
    """
    def __call__(self, model, output_path: str = None, epoch: int = -1, steps: int = -1, *args, **kwargs) -> float:
        if epoch != -1:
            out_txt = " after epoch {}:".format(epoch) if steps == -1 else " in epoch {} after {} steps:".format(epoch, steps)
        else:
            out_txt = ":"

        #logger.info("Information Retrieval Evaluation on " + self.name + " dataset" + out_txt)

        scores = self.compute_metrices(model, *args, **kwargs)

        # Write results to disc
        if output_path is not None and self.write_csv:
            csv_path = os.path.join(output_path, self.csv_file)
            if not os.path.isfile(csv_path):
                fOut = open(csv_path, mode="w", encoding="utf-8")
                fOut.write(",".join(self.csv_headers))
                fOut.write("\n")

            else:
                fOut = open(csv_path, mode="a", encoding="utf-8")

            output_data = [epoch, steps]
            for name in self.score_function_names:
                for k in self.accuracy_at_k:
                    output_data.append(scores[name]['accuracy@k'][k])

                for k in self.precision_recall_at_k:
                    output_data.append(scores[name]['precision@k'][k])
                    output_data.append(scores[name]['recall@k'][k])

                for k in self.mrr_at_k:
                    output_data.append(scores[name]['mrr@k'][k])

                for k in self.ndcg_at_k:
                    output_data.append(scores[name]['ndcg@k'][k])

                for k in self.map_at_k:
                    output_data.append(scores[name]['map@k'][k])

            fOut.write(",".join(map(str, output_data)))
            fOut.write("\n")
            fOut.close()

        if self.main_score_function is None:
            return max([scores[name]['recall@k'][max(self.precision_recall_at_k)] for name in self.score_function_names])
        else:
            return scores[self.main_score_function]['recall@k'][max(self.precision_recall_at_k)]

            
# ===============================================================
#  run
# ===============================================================
        
def run(cfg, train_examples, queries, corpus, relevant_docs):
    # model
    print(f"========== finetuning fold{cfg.trn_fold} ==========")
    word_embedding_model = models.Transformer(cfg.model, max_seq_length=cfg.max_len)
    pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), pooling_mode='mean')
    model = CustomTransformer(modules=[word_embedding_model, pooling_model])
    
    # train
    train_dataloader =  datasets.NoDuplicatesDataLoader(train_examples, batch_size=cfg.batch_size)
    train_loss = losses.MultipleNegativesSymmetricRankingLoss(model)
    warmup_steps = math.ceil(len(train_dataloader) * cfg.epochs * 0.1)

    earlystopping = EarlyStopping(patience=cfg.patience)
    
    # evaluation
    evalu = CustomEvaluator(queries, corpus, relevant_docs, corpus_chunk_size = 10000,
                                                     precision_recall_at_k = [50],
                                                     show_progress_bar = True,
                                                     batch_size = cfg.batch_size, 
                                                     name= "Evalution", 
                                                     write_csv = True, 
                                                     score_functions = {'cos_sim': cos_sim, 'dot_score': dot_score}
                                                     )
    
    model.fit(cfg,
            train_objectives=[(train_dataloader, train_loss)],
            evaluator=evalu,
            epochs=cfg.epochs,
            steps_per_epoch=cfg.steps_per_epoch,
            scheduler="warmupcosine",
            warmup_steps=warmup_steps,
            output_path=cfg.output_dir+cfg.model.replace("/", "-")+f"_fine-tuned",
            save_best_model=True,
            use_amp=True,
            show_progress_bar=False,
            callback=earlystopping
            )
    
# ===============================================================
#  main
# ===============================================================
def main(cfg):
    df_content, df_train, df_valid = data_load(cfg)
    train_examples, queries, corpus, relevant_docs = prepare_df(df_content, df_train, df_valid)
    del df_content, df_train, df_valid
    run(cfg, train_examples, queries, corpus, relevant_docs)
    
# ===============================================================
#  Execution
# ===============================================================
if __name__ == "__main__":
    args = parse_args()
    args.output_dir = args.output_dir+args.filename+"/fold_"+str(args.trn_fold)+"/1st/"
    if not os.path.exists(args.output_dir):
        os.makedirs(args.output_dir)
    for k, v in vars(args).items():
        print(f"{k}: {v}")
    main(args)

Writing 1st_stage_model_train.py


In [ ]:
!python 1st_stage_model_train.py\
--model sentence-transformers/all-mpnet-base-v2\
--batch_size 256

2023-03-02 15:50:05.478945: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-02 15:50:05.479061: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-02 15:50:05.479082: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
device: cuda
seed: 42
input_dir: /content/drive/MyDrive/KAGGLE-LECR/
output_dir: /content/drive/MyDrive/KAGGLE-LECR/exp006/fold_0/1st/
model: sentence-transformers/all

In [ ]:
from google.colab import runtime
runtime.unassign()